# Assignment Algorithms : Part 1

The purpose of this notebook is to explore the fiberassign algorithms and the consequences of introducing different target populations and realistic nominal positioner motions and exclusion zones.  Very often people see behavior of the fiberassign code which does not match their intuition and expectations.  Usually these tests also use real target data, which further complicates the situation and makes it impossible to determine what effects are due to the target data which effects are just due to geometry.

The goal of this notebook is to examine the impacts of the just the DESI geometry applied to synthetic targets with different densities and priorities.

For all these cells, we are using a single RA / DEC location on the sky and one or more tiles.  To give a more representative spread on our results we do 100 realizations of the simulation for each exercise.

## Imports and Definitions

These are global imports and variables used throughout the notebook.

In [ ]:
import os
import sys

from collections import OrderedDict

import subprocess as sp

import numpy as np

import matplotlib.pyplot as plt

from IPython.display import Image, display

import fitsio

from desitarget.targetmask import desi_mask

from fiberassign import __version__ as fba_version

from fiberassign.hardware import (
    load_hardware,
)

from fiberassign.targets import (
    Targets,
    TargetTree,
    TargetsAvailable,
    LocationsAvailable,
    load_target_table, 
    TARGET_TYPE_SCIENCE, 
    TARGET_TYPE_SKY,
    TARGET_TYPE_SUPPSKY,
    TARGET_TYPE_STANDARD
)

from fiberassign.tiles import (
    load_tiles,
)

from fiberassign.assign import (
    Assignment,
)

from fiberassign.vis import (
    plot_assignment_tile,
)

# Date used for the focalplane model
assign_date = "2020-01-01T00:00:00"

# This is a small patch around one DESI pointing
patch_ra_min = 148.0
patch_ra_max = 152.0
patch_dec_min = 29.0
patch_dec_max = 33.0

# Tile center (for our co-incident tiles)
tile_ra = 150.0
tile_dec = 31.0

# Target densities (table 3.1 of DSR)
target_density = {
    "ELG": 2400,
    "LRG": 350,
    "QSO-tracer": 170,
    "QSO-lyman": 90,
    "FAKE-high": 4000,
    "standards": 300,
    "sky": 4000
}

# DESITARGET defs
target_bitname = {
    "ELG": "ELG",
    "LRG": "LRG",
    "QSO-tracer": "QSO",
    "QSO-lyman": "QSO",
    "FAKE-high": "ELG",
    "standards": "STD_WD",
    "sky": "SKY"
}

# Target priorities
target_priority = {
    "ELG": 3000,
    "LRG": 3200,
    "QSO-tracer": 3400,
    "QSO-lyman": 3400,
    "FAKE-high": 1000,
    "standards": 0,
    "sky": 0
}

# Target requested number of observations
target_numobs = {
    "ELG": 1,
    "LRG": 1,
    "QSO-tracer": 1,
    "QSO-lyman": 4,
    "FAKE-high": 1,
    "standards": 0,
    "sky": 0
}

# Plotting color
target_pltcolor = {
    "ELG": (0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
    "LRG": (1.0, 0.4980392156862745, 0.054901960784313725),
    "QSO-tracer": (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
    "QSO-lyman": (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
    "FAKE-high": (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
    "standards": (1.0, 0.832, 0.0),
    "sky": (0.0, 1.0, 1.0)
}

In [ ]:
# Working directory (where input and output files will be written).  Change this to wherever you like.
workdir = os.path.join(os.environ["HOME"], "scratch", "desi", "tutorials", "fiberassign_part1")
# workdir = os.path.join(os.environ["SCRATCH"], "desi", "tutorials", "fiberassign_part1")

## Helper Functions

Here we define some functions to simulate tiles and uniform random target distributions, as well as some functions to help with plotting later.

In [ ]:
def sim_tiles(path, ntile):
    """Function to generate some co-incident tiles.
    """
    tile_dtype = np.dtype([
        ("TILEID", "i4"),
        ("RA", "f8"),
        ("DEC", "f8"),
        ("IN_DESI", "i4"),
        ("PROGRAM", "S6"),
        ("OBSCONDITIONS", "i4")
    ])
    fdata = np.zeros(ntile, dtype=tile_dtype)
    for i in range(ntile):
        fdata[i] = (1234+i, tile_ra, tile_dec, 1, "DARK", 1)

    if os.path.isfile(path):
        os.remove(path)
    fd = fitsio.FITS(path, "rw")

    header = dict()
    header["FBAVER"] = fba_version
    fd.write(fdata, header=header)

    return


def sim_targets(tgtype, tgoffset, path=None, density=5000.0, priority=0, numobs=0, tgbits=0):
    target_cols = OrderedDict([
        ("TARGETID", "i8"),
        ("RA", "f8"),
        ("DEC", "f8"),
        ("DESI_TARGET", "i8"),
        ("BRICKID", "i4"),
        ("BRICK_OBJID", "i4"),
        ("BRICKNAME", "a8"),
        ("PRIORITY", "i4"),
        ("SUBPRIORITY", "f8"),
        ("OBSCONDITIONS", "i4"),
        ("NUMOBS_MORE", "i4"),
        ("FIBERFLUX_G", "f4"),
        ("FIBERFLUX_R", "f4"),
        ("FIBERFLUX_Z", "f4"),
        ("FIBERFLUX_IVAR_G", "f4"),
        ("FIBERFLUX_IVAR_R", "f4"),
        ("FIBERFLUX_IVAR_Z", "f4")
    ])

    target_dtype = np.dtype([(x, y) for x, y in target_cols.items()])

    ndim = np.sqrt(density)
    nra = int(ndim * (patch_ra_max - patch_ra_min))
    ndec = int(ndim * (patch_dec_max - patch_dec_min))
    ntarget = nra * ndec

    fdata = np.zeros(ntarget, dtype=target_dtype)
    fdata["TARGETID"][:] = tgoffset + np.arange(ntarget)
    fdata["RA"][:] = np.random.uniform(low=patch_ra_min, high=patch_ra_max, size=ntarget)
    fdata["DEC"][:] = np.random.uniform(low=patch_dec_min, high=patch_dec_max, size=ntarget)
    fdata["OBSCONDITIONS"][:] = np.ones(ntarget, dtype=np.int32)
    fdata["SUBPRIORITY"][:] = np.random.uniform(low=0.0, high=1.0, size=ntarget)

    sky_mask = desi_mask["SKY"].mask
    suppsky_mask = desi_mask["SUPP_SKY"].mask
    std_mask = desi_mask["STD_BRIGHT"].mask

    if tgtype == TARGET_TYPE_SKY:
        fdata["PRIORITY"][:] = np.zeros(ntarget, dtype=np.int32)
        fdata["DESI_TARGET"][:] |= sky_mask
    elif tgtype == TARGET_TYPE_STANDARD:
        fdata["PRIORITY"][:] = priority * np.ones(ntarget, dtype=np.int32)
        fdata["DESI_TARGET"][:] |= std_mask
    elif tgtype == TARGET_TYPE_SUPPSKY:
        fdata["PRIORITY"][:] = np.zeros(ntarget, dtype=np.int32)
        fdata["DESI_TARGET"][:] |= suppsky_mask
    elif tgtype == TARGET_TYPE_SCIENCE:
        fdata["PRIORITY"][:] = priority * np.ones(ntarget, dtype=np.int32)
        fdata["NUMOBS_MORE"][:] = numobs
        fdata["DESI_TARGET"][:] |= desi_mask[tgbits].mask
    else:
        raise RuntimeError("unknown target type")

    if path is None:
        # Just return the data table
        return fdata
    else:
        # We are writing the output to a file.  Return the number of targets.
        if os.path.isfile(path):
            os.remove(path)
        fd = fitsio.FITS(path, "rw")
        header = dict()
        header["FBAVER"] = fba_version
        fd.write(fdata, header=header)
        return ntarget


def plot_assignment_stats(
        wd, outname, title, tile_ids, classes,
        hist_tgassign, hist_tgavail, hist_tgconsid
):
    fiberbins = 0.01 * np.arange(101)
    tgbins = {
        "ELG": np.arange(0, 22220, 220),
        "LRG": np.arange(0, 3232, 32),
        "QSO-tracer": np.arange(0, 1515, 15),
        "QSO-lyman": np.arange(0, 808, 8),
        "FAKE-high": np.arange(0, 36360, 360),
        "standards": np.arange(0, 3636, 36),
        "sky": np.arange(0, 40400, 400)
    }
    
    for tid in tile_ids:
        # The plots
        figfiber = plt.figure(figsize=(12, 6))
        figtarget = plt.figure(figsize=(12, 12))
        
        # Plot the positioner assignment fractions
        axfiber = figfiber.add_subplot(1, 1, 1)
        
        for tgclass in classes:
            if np.sum(hist_tgassign[tid][tgclass]) > 0:
                axfiber.hist(
                    np.array(hist_tgassign[tid][tgclass]) / 5000, 
                    fiberbins, 
                    align="mid", 
                    alpha=0.8, 
                    label="{}".format(tgclass),
                    color=target_pltcolor[tgclass]
                )
        axfiber.set_xlabel("Fraction of Assigned Positioners", fontsize="large")
        axfiber.set_ylabel("Realization Counts", fontsize="large")
        axfiber.set_ylim(0, 100)
        axfiber.set_title(
            "Tile {}:  Positioner Assignment : {}".format(tid, title)
        )
        axfiber.legend()
        figfiber.savefig(
            os.path.join(wd, "fiberassign_{}_fibers_tile-{}.pdf".format(outname, tid)),
            dpi=300, 
            format="pdf"
        )
        figfiber.show()
            
        nplot = len(classes)
        pcols = 2
        prows = (nplot + 1) // pcols
        poff = 1
        for tgclass in classes:
            if np.sum(hist_tgavail[tid][tgclass]) == 0:
                continue
            axtarget = figtarget.add_subplot(prows, pcols, poff)
            axtarget.hist(
                np.array(hist_tgavail[tid][tgclass]), 
                tgbins[tgclass], 
                align="mid", 
                alpha=0.4, 
                label="{} Reachable".format(tgclass),
                color=(0.7, 0.7, 0.7)
            )
            axtarget.hist(
                np.array(hist_tgconsid[tid][tgclass]), 
                tgbins[tgclass], 
                align="mid", 
                alpha=0.4, 
                label="{} Considered".format(tgclass),
                color=target_pltcolor[tgclass]
            )
            axtarget.hist(
                np.array(hist_tgassign[tid][tgclass]), 
                tgbins[tgclass], 
                align="mid", 
                alpha=0.8, 
                label="{} Assigned".format(tgclass),
                color=target_pltcolor[tgclass]
            )
            axtarget.set_xlabel("Targets Reachable, Considered, and Assigned", fontsize="large")
            axtarget.set_ylabel("Realization Counts", fontsize="large")
            axtarget.set_ylim(0, 100)
            axtarget.legend()
            poff += 1
        
        figtarget.suptitle(
            "Tile {}:  {}".format(tid, title), fontsize="x-large"
        )
        figtarget.tight_layout(rect=[0, 0, 1, 0.97])
        figtarget.savefig(
            os.path.join(wd, "fiberassign_{}_targets_tile-{}.pdf".format(outname, tid)),
            dpi=300, 
            format="pdf"
        )
        figtarget.show()

        
def create_histogram_dicts(tiles, classes):
    hist_tgassign = dict()
    hist_tgconsid = dict()
    hist_tgavail = dict()
    for tid in tiles:
        hist_tgassign[tid] = dict()
        hist_tgavail[tid] = dict()
        hist_tgconsid[tid] = dict()
        for tgclass in classes:
            hist_tgassign[tid][tgclass] = list()
            hist_tgavail[tid][tgclass] = list()
            hist_tgconsid[tid][tgclass] = list()
    return (hist_tgassign, hist_tgavail, hist_tgconsid)


def accum_histogram_data(
    tile, classes, tile_assign, tile_avail, id2class, tgobs, 
    hist_tgassign, hist_tgavail, hist_tgconsid, verbose=False
):      
    # The locations that were assigned to a target
    locs_assigned = [x for x, y in tile_assign.items() if y >= 0]
    
    if verbose:
        print(
            "    tile {}:  {} positioners ({:0.1f}%) assigned a target".format(
                tile, len(locs_assigned), 100*(len(locs_assigned)/5000)
            )
        )

    # Compute the numbers of each target class that were assigned and available
    assign_by_class = dict()
    avail_by_class = dict()
    consider_by_class = dict()

    for tgclass in classes:
        assign_by_class[tgclass] = list()
        avail_by_class[tgclass] = list()
        consider_by_class[tgclass] = list()

    for loc in locs_assigned:
        tgid = tile_assign[loc]
        tgstr = id2class(tgid)
        assign_by_class[tgstr].append(tgid)
        for avtg in tile_avail[loc]:
            avtgstr = id2class(avtg)
            avail_by_class[avtgstr].append(avtg)
            if avtgstr == "standards" or avtgstr == "sky" or tgobs[avtg] > 0:
                # This target was actually considered for assignment
                consider_by_class[avtgstr].append(avtg)
    
    # Decrement the obs remaining for the assigned targets
    for loc in locs_assigned:
        tgid = tile_assign[loc]
        tgobs[tgid] -= 1

    # Accumulate to the target histograms
    for tgclass in classes:
        hist_tgassign[tid][tgclass].append(len(assign_by_class[tgclass]))
        uniq = np.unique(np.array(avail_by_class[tgclass]))
        hist_tgavail[tid][tgclass].append(len(uniq))
        uniq = np.unique(np.array(consider_by_class[tgclass]))
        hist_tgconsid[tid][tgclass].append(len(uniq))
    return
    

## Focalplane Coverage

The hardware information that fiberassign gets from the `desimodel` package contains the positioner arm lengths and ranges of motion for all positioners.  We can use that along with a random sampling of locations to illustrate the regions that are not reachable by any positioner.

In [ ]:
# Directory for this section
wdir = os.path.join(workdir, "focalplane")
os.makedirs(wdir, exist_ok=True)

# Set the random seed to ensure reproducibility of this cell
np.random.seed(123456)

# Read hardware properties
test_date = assign_date
# If you uncomment this line, you will get the commissioning
# focalplane with restricted positioner reach.
#test_date = "2020-04-01T00:00:00"
hw = load_hardware(rundate=test_date)

# Simulate a single tile and load
tfile = os.path.join(wdir, "footprint.fits")
sim_tiles(tfile, 1)
tiles = load_tiles(tiles_file=tfile)

# Generate target table at high density
tgdensity = 10000
tgdata = sim_targets(
    TARGET_TYPE_SCIENCE, 
    0,
    density=tgdensity,
    priority=0,
    numobs=0,
    tgbits="SKY"
)
print("{} fake targets".format(len(tgdata)))
tgs = Targets()
load_target_table(tgs, tgdata)

# Compute the targets available to all positioners
tree = TargetTree(tgs, 0.01)
tgsavail = TargetsAvailable(hw, tgs, tiles, tree)
del tree

# Availability for this single tile
tid = tiles.id[0]
tavail = tgsavail.tile_data(tid)

# Get the unique set of target IDs reachable by *any* positioner
reachable = np.unique([x for loc, tgl in tavail.items() for x in tgl])
print("{} reachable targets".format(len(reachable)))

# Now get the rows of the original target table that are NOT in this set
unreachable_rows = np.where(
    np.isin(tgdata["TARGETID"], reachable, invert=True)
)[0]
print("{} un-reachable targets".format(len(unreachable_rows)))

# Now we would like to plot the targets that were not reachable.  Also
# plot the positioner angle ranges.

locs = np.array(hw.locations)
loc_theta_offset = hw.loc_theta_offset
loc_theta_min = hw.loc_theta_min
loc_theta_max = hw.loc_theta_max
loc_phi_offset = hw.loc_phi_offset
loc_phi_min = hw.loc_phi_min
loc_phi_max = hw.loc_phi_max

theta_off = np.array([loc_theta_offset[x]*180/np.pi for x in locs])
theta_min = np.array([loc_theta_min[x]*180/np.pi for x in locs])
theta_max = np.array([loc_theta_max[x]*180/np.pi for x in locs])

phi_off = np.array([loc_phi_offset[x]*180/np.pi for x in locs])
phi_min = np.array([loc_phi_min[x]*180/np.pi for x in locs])
phi_max = np.array([loc_phi_max[x]*180/np.pi for x in locs])

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 2, 1)
ax.scatter(locs, theta_off, marker="o", s=0.5, label="theta off")
ax.scatter(locs, theta_min, marker="o", s=0.5, label="theta min")
ax.scatter(locs, theta_max, marker="o", s=0.5, label="theta max")
ax.set_xlabel("Positioner Location", fontsize="large")
ax.set_ylabel("Degrees", fontsize="large")
ax.legend()

ax = fig.add_subplot(1, 2, 2)
ax.scatter(locs, phi_off, marker="o", s=0.5, label="phi off")
ax.scatter(locs, phi_min, marker="o", s=0.5, label="phi min")
ax.scatter(locs, phi_max, marker="o", s=0.5, label="phi max")
ax.set_xlabel("Positioner Location", fontsize="large")
ax.set_ylabel("Degrees", fontsize="large")
ax.legend()
plt.show()

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
ax.set_aspect("equal")

center_mm = hw.loc_pos_curved_mm
theta_arm = hw.loc_theta_arm
phi_arm = hw.loc_phi_arm
patrol_buffer = hw.patrol_buffer_mm
    
for loc in hw.locations:
    patrol_rad = theta_arm[loc] + phi_arm[loc] - patrol_buffer
    patrol = plt.Circle(
        (center_mm[loc][0], center_mm[loc][1]), 
        radius=patrol_rad, 
        fc="red",
        ec="none", 
        alpha=0.1
    )
    ax.add_artist(patrol)

tgxy = hw.radec2xy_multi(
    tiles.ra[0], 
    tiles.dec[0],
    tiles.obstheta[0],
    tgdata["RA"][unreachable_rows],
    tgdata["DEC"][unreachable_rows],
    False,
    1
)
tgx = np.array([x[0] for x in tgxy])
tgy = np.array([x[1] for x in tgxy])

ax.scatter(
    tgx,
    tgy,
    color=(0.6, 0.6, 0.6),
    marker=".",
    s=0.01
)

ax.set_xlabel("Millimeters", fontsize="large")
ax.set_ylabel("Millimeters", fontsize="large")
pfile = os.path.join(wdir, "coverage_{}.pdf".format(tgdensity))
plt.savefig(pfile, dpi=300, format="pdf")
plt.close()

print("PDF written to {}\n  Displaying low-res image inline...".format(pfile))
png_file = os.path.join(wdir, "coverage_{}.png".format(tgdensity))
cmd = 'convert -density 200 {} {}'.format(pfile, png_file)
sp.check_call(cmd, stderr=sp.STDOUT, shell=True)
img = Image(filename=(png_file))
display(img)

## Effects of Positioner Geometry

The exclusion zone around each positioner and its range of motion leads to an average limit on the fraction of targets that can be assigned for a given target density.  We can see this intrinsic limit by simulating realizations of a single population of random targets for several densities. The plots of the positioner assignments for the first realization can take a while- you can comment out that if you like.

In [ ]:
# Directory for this section
wdir = os.path.join(workdir, "geometry")
os.makedirs(wdir, exist_ok=True)

# Set the random seed to ensure reproducibility of this cell
np.random.seed(123456)

# Read hardware properties
hw = load_hardware(rundate=assign_date)

# Simulate a single tile and load
tfile = os.path.join(wdir, "footprint.fits")
sim_tiles(tfile, 1)
tiles = load_tiles(tiles_file=tfile)

# Accumulation dictionaries
(hist_tgassign, hist_tgavail, hist_tgconsid) = \
    create_histogram_dicts(tiles.id, list(target_density.keys()))

# Consider these 5 classes.
target_classes = ["ELG", "LRG", "QSO-tracer", "QSO-lyman", "FAKE-high"]

for tgclass in target_classes:
    tgdensity = target_density[tgclass]

    print("Working on target density {}...".format(tgdensity))
    
    # For each realization...
    for mc in range(100):
        # Generate target table
        tgdata = sim_targets(
            TARGET_TYPE_SCIENCE, 
            0,
            density=tgdensity,
            priority=target_priority[tgclass],
            numobs=target_numobs[tgclass],
            tgbits=target_bitname[tgclass]
        )
        tgs = Targets()
        load_target_table(tgs, tgdata)
        
        # Make a working copy of the obs remaining for all targets, so we can decrement
        # and track the targets that were considered (which had obsremain > 0)
        tg_obs = dict()
        for tgid in tgs.ids():
            tgprops = tgs.get(tgid)
            tg_obs[tgid] = tgprops.obsremain

        # Create a hierarchical triangle mesh lookup of the targets positions
        tree = TargetTree(tgs, 0.01)

        # Compute the targets available to each fiber for each tile.
        tgsavail = TargetsAvailable(hw, tgs, tiles, tree)

        # Free the tree
        del tree

        # Compute the fibers on all tiles available for each target
        favail = LocationsAvailable(tgsavail)

        # Create assignment object
        asgn = Assignment(tgs, tgsavail, favail)

        # assignment of science targets
        asgn.assign_unused(TARGET_TYPE_SCIENCE)
        
        # Targets available for all tile / locs
        tgsavail = asgn.targets_avail()
        
        # Get the assignment for this one tile
        tid = tiles.id[0]
        tassign = asgn.tile_location_target(tid)
        
        # Targets available for this tile
        tavail = tgsavail.tile_data(tid)
        
        # Plot the first realization
        # THIS IS SLOW- after doing it once you can comment it out to save time.
#         if mc == 0:
#             pfile = os.path.join(wdir, "assignment_density_{}.pdf".format(tgdensity))
#             #print(locs_assigned, flush=True)
#             plot_assignment_tile(
#                 hw, tgs, tiles.id[0], tiles.ra[0], tiles.dec[0], tiles.obstheta[0],
#                 tassign, tile_avail=tgsavail.tile_data(tiles.id[0]), real_shapes=True, 
#                 outfile=pfile, figsize=8
#             )
#             print("Full resolution PDF written to {}, displaying low-res image inline...".format(pfile))
#             png_file = os.path.join(wdir, "assignment_density_{}.png".format(tgdensity))
#             cmd = 'convert -density 200 {} {}'.format(pfile, png_file)
#             sp.check_call(cmd, stderr=sp.STDOUT, shell=True)
#             img = Image(filename=(png_file))
#             display(img)
        
        # Accumulate results
        
        def target_id_class(target_id):
            # We are doing one class at a time in this example
            return tgclass
        
        accum_histogram_data(
            tid, [tgclass], tassign, tavail, target_id_class, 
            tg_obs, hist_tgassign, hist_tgavail, hist_tgconsid
        )

In [ ]:
# Plot it

plot_assignment_stats(
    wdir,
    "geometry", 
    "One Target Class at a Time", 
    [tiles.id[0]], 
    list(target_density.keys()), 
    hist_tgassign, 
    hist_tgavail, 
    hist_tgconsid
)

The previous plots are not "surprising".  They simply show that for low (QSO-like) target densities there are fewer collisions and a larger fraction (80-90%) of targets can be assigned, even though most positioners do not get one of these targets.  For a medium (LRG-like) target density we have more collisions and around 70% of available targets can be assigned while slightly less than half of the positioners get a target.  Going to a higher (ELG-like) density we have far more targets than can be assigned, but there are sufficient numbers that nearly every positioner gets a target.

**NOTE:  remember that these are uniform spatial target distributions with no physically realistic clustering.  Clustering of targets reduces the fraction that can be assigned, due to positioner collisions.**

## Effects of Multiple Target Classes

The previous section looked at different target densities but assigned each one in isolation.  Now we examine the consequences of having 3 target classes with different priorities (and densities).  The goal of this section is to demonstrate how priorities affect the resulting assignment.  The target locations are still spatially uniform with no clustering.  We will also demonstrate explicitly that low priority targets never "take" positioners from high priority targets.

In the case of two QSO populations, these have the same `PRIORITY` value.  However the fiberassign code currently computes an effective "total priority" for each target that is a combination of the (integer) `PRIORITY`, the (random from 0.0-1.0) `SUBPRIORITY` and the number of remaining observations.  See the discussion here: https://github.com/desihub/fiberassign/issues/196 .  For now, targets with the same `PRIORITY` and a larger number of observations remaining are given a higher "total priority". 

In [ ]:
# Directory for this section
wdir = os.path.join(workdir, "target_classes")
os.makedirs(wdir, exist_ok=True)

# Set the random seed to ensure reproducibility of this cell
np.random.seed(123456)

# Read hardware properties
hw = load_hardware(rundate=assign_date)

# Simulate a single tile and load
tfile = os.path.join(wdir, "footprint.fits")
sim_tiles(tfile, 1)
tiles = load_tiles(tiles_file=tfile)

# Consider these 4 classes.
target_classes = ["ELG", "LRG", "QSO-tracer", "QSO-lyman"]

# Accumulation dictionaries
(hist_tgassign, hist_tgavail, hist_tgconsid) = \
    create_histogram_dicts(tiles.id, target_classes)

# For each realization...
for mc in range(100):

    # Generate target tables separately for each target class so we can combine them later.
    tg_tables = dict()
    tgid_offset = dict()
    ntarget = dict()
    toff = 0
    for tgclass in target_classes:
        tg_tables[tgclass] = sim_targets(
            TARGET_TYPE_SCIENCE, 
            toff,
            density=target_density[tgclass],
            priority=target_priority[tgclass],
            numobs=target_numobs[tgclass],
            tgbits=target_bitname[tgclass]
        )
        tgid_offset[tgclass] = toff
        ntarget[tgclass] = len(tg_tables[tgclass])
        toff += ntarget[tgclass]
        
    def target_id_class(target_id):
        """Helper function to go from target ID back to the class"""
        for cls in target_classes:
            if (target_id >= tgid_offset[cls]) and (target_id < tgid_offset[cls] + ntarget[cls]):
                return cls
        return None

    # Assign just the QSOs separately, and save for comparison.
    
    qso_tgs = Targets()
    load_target_table(qso_tgs, tg_tables["QSO-tracer"])
    load_target_table(qso_tgs, tg_tables["QSO-lyman"])

    tree = TargetTree(qso_tgs, 0.01)
    tgsavail = TargetsAvailable(hw, qso_tgs, tiles, tree)
    del tree
    favail = LocationsAvailable(tgsavail)
    asgn = Assignment(qso_tgs, tgsavail, favail)
    asgn.assign_unused(TARGET_TYPE_SCIENCE)
    tid = tiles.id[0]
    qso_assign = dict(asgn.tile_location_target(tid))

    # Now assign all targets

    all_tgs = Targets()
    for tgclass in target_classes:
        load_target_table(all_tgs, tg_tables[tgclass])
        
    # Make a working copy of the obs remaining for all targets, so we can decrement
    # and track the targets that were considered (which had obsremain > 0)
    tg_obs = dict()
    for tgid in all_tgs.ids():
        tgprops = all_tgs.get(tgid)
        tg_obs[tgid] = tgprops.obsremain

    tree = TargetTree(all_tgs, 0.01)
    tgsavail = TargetsAvailable(hw, all_tgs, tiles, tree)
    del tree
    favail = LocationsAvailable(tgsavail)
    asgn = Assignment(all_tgs, tgsavail, favail)
    asgn.assign_unused(TARGET_TYPE_SCIENCE)
    tid = tiles.id[0]
    all_assign = asgn.tile_location_target(tid)

    print(
        "Realization {:02d}:  assigning all targets:  {} positioners assigned".format(
            mc, 
            int(np.sum([1 for x, y in all_assign.items() if y >= 0]))
        )
    )

    # Verify that QSO assignment for all targets is identical to the case of assigning only the QSOs
    qso_pos = [x for x, y in qso_assign.items() if y >= 0]
    mismatch = int(np.sum([1 for x, y in qso_assign.items() if all_assign[x] != y]))
    if mismatch > 0:
        print(
            "Realization {:02d}:    WARNING: {} positioners from QSO-only assignment changed!".format(
                mc, 
                mismatch
            )
        )
    
    # Targets available for all tile / locs
    tgsavail = asgn.targets_avail()

    # available for this tile
    avail = tgsavail.tile_data(tid)
    
    # Accumulate results
    accum_histogram_data(
        tid, target_classes, all_assign, avail, target_id_class, 
        tg_obs, hist_tgassign, hist_tgavail, hist_tgconsid,
        verbose=False
    )


In [ ]:
# Plot the results

plot_assignment_stats(
    wdir,
    "allscience", 
    "All Science Classes for One Tile", 
    [tiles.id[0]], 
    target_classes, 
    hist_tgassign, 
    hist_tgavail, 
    hist_tgconsid
)


The exercise above shows several things:

1.  The highest "total priority" targets (which also have a low density) are assigned first and nearly all get assigned to a fiber.

2.  As the target "total priority" decreases (and the density increases), these targets take up larger fractions of the positioners.

3.  During this first-pass assignment, about 98% of positioners get assigned to a science target (remember we are not assigning standards and sky in this excercise).

4.  The introduction of lower priority targets **does not** affect the assignment of higher priority targets.

Compare the "Fraction of Assigned Positioners" plot to the one from the previous section.  You can see that the highest priority class has the same assignment fraction, since it is assigned first.  The other target classes have smaller fractions of the positioners because they are forced to use only positioners leftover after the higher priority targets are assigned.

## Effects of Multiple Passes

So far we have worked with one tile.  However the QSO-lyman population has 4 requested observations.  If we have multiple observations of the same point on the sky, we would expect these targets to continue to get preferential assignments until their observations remaining equals "1" at which point they have the same observations remaining as the other QSO-tracer population.  At that point the random subriority will determine the total priority ordering.  And of course positioners will continue to be assigned to all target classes as possible.

In the real survey, tiles from multiple "layers" (passes over the footprint) are dithered so that they do not overlap perfectly.  For this study of one pointing on the sky, we will use 4 tiles (passes) centered on the same RA / DEC.

In [ ]:
# Directory for this section
wdir = os.path.join(workdir, "multipass")
os.makedirs(wdir, exist_ok=True)

# Set the random seed to ensure reproducibility of this cell
np.random.seed(123456)

# Read hardware properties
hw = load_hardware(rundate=assign_date)

# Simulate 5 coincident tiles and load
tfile = os.path.join(wdir, "footprint.fits")
sim_tiles(tfile, 4)
tiles = load_tiles(tiles_file=tfile)

# Consider these 4 classes.
target_classes = ["ELG", "LRG", "QSO-tracer", "QSO-lyman"]

# Accumulation dictionaries
(hist_tgassign, hist_tgavail, hist_tgconsid) = \
    create_histogram_dicts(tiles.id, target_classes)

# For each realization...
for mc in range(100):

    # Generate target tables
    tg_tables = dict()
    tgid_offset = dict()
    ntarget = dict()
    toff = 0
    for tgclass in target_classes:
        tg_tables[tgclass] = sim_targets(
            TARGET_TYPE_SCIENCE, 
            toff,
            density=target_density[tgclass],
            priority=target_priority[tgclass],
            numobs=target_numobs[tgclass],
            tgbits=target_bitname[tgclass]
        )
        tgid_offset[tgclass] = toff
        ntarget[tgclass] = len(tg_tables[tgclass])
        toff += ntarget[tgclass]
        
    def target_id_class(target_id):
        for cls in target_classes:
            if (target_id >= tgid_offset[cls]) and (target_id < tgid_offset[cls] + ntarget[cls]):
                return cls
        return None

    # Load the targets

    all_tgs = Targets()
    for tgclass in target_classes:
        load_target_table(all_tgs, tg_tables[tgclass])
        
    # Make a working copy of the obs remaining for all targets, so we can decrement
    # and track the targets that were considered (which had obsremain > 0)
    tg_obs = dict()
    for tgid in all_tgs.ids():
        tgprops = all_tgs.get(tgid)
        tg_obs[tgid] = tgprops.obsremain

    tree = TargetTree(all_tgs, 0.01)
    tgsavail = TargetsAvailable(hw, all_tgs, tiles, tree)
    del tree
    favail = LocationsAvailable(tgsavail)
    asgn = Assignment(all_tgs, tgsavail, favail)
    asgn.assign_unused(TARGET_TYPE_SCIENCE)
    
    print("Realization {:02d}:".format(mc))
    
    # Accumulate results for all tiles
    for tid in tiles.id:
        # Assignment for this tile
        tile_assign = asgn.tile_location_target(tid)

        # Targets available for all tile / locs
        tgsavail = asgn.targets_avail()

        # available for this tile
        tile_avail = tgsavail.tile_data(tid)
        
        # Accumulate results
        accum_histogram_data(
            tid, target_classes, tile_assign, tile_avail, target_id_class, 
            tg_obs, hist_tgassign, hist_tgavail, hist_tgconsid,
            verbose=True
        )


In [ ]:
# Plot the results

plot_assignment_stats(
    wdir,
    "multipass", 
    "All Science Classes for {} Passes".format(len(tiles.id)), 
    tiles.id, 
    target_classes, 
    hist_tgassign, 
    hist_tgavail, 
    hist_tgconsid
)


As expected, the results from the first pass are identical to the previous exercise.  This is because all targets have at least one requested observation.  In the subsequent passes, we see the impact of some targets having met their required observations.  Although the numbers of "reachable" targets remains stable, we can see that the number of targets "considered" for assignment (i.e. that have remaining observations) goes down with subsequent tiles.  This is most notable with the QSO-lyman target class that has 4 requested observations per target.

## Sky and Standards

After assigning science targets, we must free up a certain number of fibers for calibration standards and sky.

In [ ]:
# Directory for this section
wdir = os.path.join(workdir, "sky-std")
os.makedirs(wdir, exist_ok=True)

# Set the random seed to ensure reproducibility of this cell
np.random.seed(123456)

# Read hardware properties
hw = load_hardware(rundate=assign_date)

# Simulate 5 coincident tiles and load
tfile = os.path.join(wdir, "footprint.fits")
sim_tiles(tfile, 4)
tiles = load_tiles(tiles_file=tfile)

# Consider these 4 classes.
target_classes = ["ELG", "LRG", "QSO-tracer", "QSO-lyman"]

all_classes = list(target_classes)
all_classes.extend(["standards", "sky"])
    
# Accumulation dictionaries
(hist_tgassign, hist_tgavail, hist_tgconsid) = \
    create_histogram_dicts(tiles.id, all_classes)

# For each realization...
for mc in range(100):

    # Generate target tables
    tg_tables = dict()
    tgid_offset = dict()
    ntarget = dict()
    toff = 0
    for tgclass in target_classes:
        tg_tables[tgclass] = sim_targets(
            TARGET_TYPE_SCIENCE, 
            toff,
            density=target_density[tgclass],
            priority=target_priority[tgclass],
            numobs=target_numobs[tgclass],
            tgbits=target_bitname[tgclass]
        )
        tgid_offset[tgclass] = toff
        ntarget[tgclass] = len(tg_tables[tgclass])
        toff += ntarget[tgclass]
    
    tg_tables["standards"] = sim_targets(
        TARGET_TYPE_STANDARD, 
        toff,
        density=target_density["standards"],
    )
    tgid_offset["standards"] = toff
    ntarget["standards"] = len(tg_tables["standards"])
    toff += ntarget["standards"]
    
    tg_tables["sky"] = sim_targets(
        TARGET_TYPE_SKY, 
        toff,
        density=target_density["sky"],
    )
    tgid_offset["sky"] = toff
    ntarget["sky"] = len(tg_tables["sky"])
    toff += ntarget["sky"]
    
    def target_id_class(target_id):
        for cls in all_classes:
            if (target_id >= tgid_offset[cls]) and (target_id < tgid_offset[cls] + ntarget[cls]):
                return cls
        return None

    # Load the targets

    all_tgs = Targets()
    for tgclass in all_classes:
        load_target_table(all_tgs, tg_tables[tgclass])
        
    # Make a working copy of the obs remaining for all targets, so we can decrement
    # and track the targets that were considered (which had obsremain > 0)
    tg_obs = dict()
    for tgid in all_tgs.ids():
        tgprops = all_tgs.get(tgid)
        tg_obs[tgid] = tgprops.obsremain

    tree = TargetTree(all_tgs, 0.01)
    tgsavail = TargetsAvailable(hw, all_tgs, tiles, tree)
    del tree
    favail = LocationsAvailable(tgsavail)
    asgn = Assignment(all_tgs, tgsavail, favail)
    asgn.assign_unused(TARGET_TYPE_SCIENCE)
    
    # Assign standards, up to some limit
    asgn.assign_unused(TARGET_TYPE_STANDARD, 10)

    # Assign sky to unused fibers, up to some limit
    asgn.assign_unused(TARGET_TYPE_SKY, 40)

    # Force assignment if needed
    asgn.assign_force(TARGET_TYPE_STANDARD, 10)
    asgn.assign_force(TARGET_TYPE_SKY, 40)
    
    print("Realization {:02d}:".format(mc))
    
    # Accumulate results for all tiles
    for tid in tiles.id:
        # Assignment for this tile
        tile_assign = asgn.tile_location_target(tid)

        # Targets available for all tile / locs
        tgsavail = asgn.targets_avail()

        # available for this tile
        tile_avail = tgsavail.tile_data(tid)
        
        # Accumulate results
        accum_histogram_data(
            tid, all_classes, tile_assign, tile_avail, target_id_class, 
            tg_obs, hist_tgassign, hist_tgavail, hist_tgconsid,
            verbose=True
        )


In [ ]:
# Plot the results

plot_assignment_stats(
    wdir,
    "sky-std", 
    "Science, Standards and Sky for {} Passes".format(len(tiles.id)), 
    tiles.id, 
    all_classes, 
    hist_tgassign, 
    hist_tgavail, 
    hist_tgconsid
)

When interpreting the above plots, first consider the "fraction of assigned positioners" for each tile.  The sharply peaked histogram for standards and sky is a result of us **forcing** 10 standards and 40 skies per petal.  When doing this "bumping" of science targets, we start with the lowest priority targets and this is why the overall number of ELGs drops for each tile compared to the previous section.

## Conclusions

Hopefully this notebook has built some intuition about the limits of DESI positioner geometry and the assignment results that occur in the presence of multiple target classes, each with different priorities and densities, and observed over multiple passes / layers.  Part 2 of this study will bring in a "real" target sample.